In [918]:
import pandas as pd

In [919]:
from datetime import datetime

In [920]:
from dateutil.relativedelta import relativedelta

## 1) Для каждого абонента определить количество всех событий (звонки/интернет-сессии/пополнения/списания)

In [939]:
temp_events = pd.read_excel("temp_events.xlsx")
event_count = temp_events[['subs_key', 'event_type']].groupby(['subs_key'], as_index=False).count()
event_count.head()

,subs_key,event_type
0,123,7
1,124,4
2,125,4
3,126,6


## Продолжительность всех вызовов

In [940]:
temp_date = temp_events[["subs_key","event_type", "start_dt","end_dt"]]
to_datetime = lambda d: pd.to_datetime(d, format='%Y%m%d-%H%M%S').astype(int)
temp_date[['start_dt','end_dt']] = temp_date[['start_dt','end_dt']].apply(pd.to_datetime)
temp_date['duration'] = (temp_date['end_dt'] - temp_date['start_dt'])
durations = temp_date.groupby(['subs_key','event_type'],as_index=False).sum()
durations.head(100)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,subs_key,event_type,duration
0,123,1,0 days 00:17:47
1,123,2,0 days 04:58:46
2,124,1,0 days 00:15:31
3,124,2,0 days 01:41:47
4,125,1,0 days 00:05:09
5,125,2,0 days 01:54:54
6,126,1,0 days 00:08:08
7,126,2,0 days 04:45:51


## Суммa пополнений

In [941]:
temp_payments = pd.read_excel("temp_payments.xlsx")
payments = temp_payments[["subs_key", "payment"]].groupby(['subs_key'],as_index=False).sum()
payments.head()

,payment
subs_key,
123,820
124,280
125,200
126,240


## Cуммa списаний

In [942]:
temp_charges = pd.read_excel("temp_charges.xlsx")
charges = temp_charges[["subs_key", "charge"]].groupby(['subs_key'],as_index=False).sum()
charges.head()

,charge
subs_key,
123,100
124,220
125,180
126,70


## 2) Определить продолжительность самого длительного вызова и самой длительной интернет-сессии. Необходимые поля: subs_Key, duration (длительность интернет-сессии / голосового вызова)

In [943]:
max_durations = durations.groupby('event_type').max()
max_durations= pd.merge(left= max_durations['duration'], right=durations[['subs_key','duration']], left_on='duration', right_on='duration')
max_durations.head(100)

,duration,subs_key
0,0 days 00:17:47,123
1,0 days 04:58:46,123


## 3)	Для каждого абонента выгрузить историю изменения его баланса (после каждого пополнения / списания). Необходимые поля: subs_key, event_dt, change (если пополнение, то значение PAYMENT, если списание, то значение (-1) * CHARGE), balance. Отсортировать строки по абонентам и по дате события

In [944]:
normalize_charge = temp_charges[["subs_key","event_dt", "charge"]]
normalize_charge["change"] = temp_charges["charge"] * (-1)
normalize_charge = normalize_charge[["subs_key","event_dt","change"]]
normalize_charge.head()

,subs_key,event_dt,change
0,123,13.12.2020 17:25:00,-50
1,123,14.12.2020 14:40:02,-50
2,124,14.12.2020 18:27:00,-220
3,125,12.12.2020 06:30:00,-60
4,125,13.12.2020 06:30:00,-60


In [945]:
normalize_payment = temp_payments[["subs_key","event_dt", "payment"]]
normalize_payment["change"] = temp_payments["payment"] 
normalize_payment = normalize_payment[["subs_key","event_dt", "change"]]
normalize_payment.head()

,subs_key,event_dt,change
0,123,12.12.2020 07:25:21,90
1,123,15.12.2020 18:31:29,700
2,123,10.12.2020 06:30:00,30
3,124,10.12.2020 20:21:22,130
4,124,15.12.2020 04:20:20,150


In [946]:
bigdata = normalize_charge.append(normalize_payment, ignore_index=True)
bidgata = bigdata.sort_values(by='event_dt', inplace=True)
bidgata = bigdata.sort_values(by='subs_key', inplace=True)
bigdata['balance'] = bigdata.change.cumsum()
bigdata.head(100)

,subs_key,event_dt,change,balance
9,123,10.12.2020 06:30:00,30,30
7,123,12.12.2020 07:25:21,90,120
0,123,13.12.2020 17:25:00,-50,70
1,123,14.12.2020 14:40:02,-50,20
8,123,15.12.2020 18:31:29,700,720
10,124,10.12.2020 20:21:22,130,850
2,124,14.12.2020 18:27:00,-220,630
11,124,15.12.2020 04:20:20,150,780
12,125,11.12.2020 17:24:55,200,980
3,125,12.12.2020 06:30:00,-60,920


# 4) Для абонентов, у которых было больше одного списания, определить, каким типом сетевого события (голос / интернет) они пользуются чаще (сравнивать не по продолжительности, а по количеству). Необходимые поля: абонент, количество списаний, «максимальный» тип сетевого события, количество сетевых событий этого тип

In [947]:
multiple_charges = temp_charges[["subs_key","charge"]].groupby('subs_key').filter(lambda x: x['subs_key'].count()>1)
multiple_charges = multiple_charges[["subs_key","charge"]].groupby(['subs_key'],as_index=False).count()
multiple_charges.head()

,subs_key,charge
0,123,2
1,125,3


In [948]:
multiple_charges= pd.merge(left= multiple_charges[['subs_key', 'charge']], right=temp_events[['subs_key','event_type']], left_on='subs_key', right_on='subs_key')
multiple_charges.head(100)

,subs_key,charge,event_type
0,123,2,1
1,123,2,1
2,123,2,1
3,123,2,2
4,123,2,2
5,123,2,2
6,123,2,1
7,125,3,1
8,125,3,1
9,125,3,1


In [949]:
frequent_event=multiple_charges.groupby(['subs_key','event_type']).size().reset_index(name='count')
frequent_event = frequent_event.groupby(['subs_key'], as_index=False).max()
frequent_event.head(100)

,subs_key,event_type,count
0,123,2,4
1,125,2,3


In [950]:
multiple_charges= pd.merge(left= multiple_charges[['subs_key', 'charge']], right=frequent_event[['subs_key','event_type','count']], left_on='subs_key', right_on='subs_key')
multiple_charges = multiple_charges.drop_duplicates(subset=None, keep="first", inplace=False)
multiple_charges.head(100)

,subs_key,charge,event_type,count
0,123,2,2,4
7,125,3,2,3
